In [1]:
import pandas as pd
# from joblib import Parallel, delayed
from tqdm import tqdm
# import torch

In [2]:

df = pd.read_csv("marathi_iitb_file_dur.csv",index_col=False)
df = df.drop(columns=["Unnamed: 0"])

In [3]:
df

,audio_filepath,duration,text
0,marathi_iitb_microsoft/College-Students_resamp...,2.68,रामटेक ते वर्धा रेल्वे ने कसं जायचं ?
1,marathi_iitb_microsoft/College-Students_resamp...,4.78,मला सांग की ऊर्जा ग्लोबल चा ग्राहक निगा क्रमां...
2,marathi_iitb_microsoft/College-Students_resamp...,4.56,सिंधुदुर्ग पॅसेंजर राजापूर रोड स्टेशन ला कधी य...
3,marathi_iitb_microsoft/College-Students_resamp...,2.58,आता या नंतर कोणतं गाणं आहे ?
4,marathi_iitb_microsoft/College-Students_resamp...,6.00,सवारी कार रेंटल्स फरीदाबाद चा तक्रार निवारण क्...
...,...,...,...
92466,marathi_iitb_microsoft/Rural-Low-Income-Worker...,7.80,मुंबई एलटीटी - पाटलिपुत्र एक्सप्रेस चे टाइम शे...
92467,marathi_iitb_microsoft/Rural-Low-Income-Worker...,1.48,शो प्रोफाईल.
92468,marathi_iitb_microsoft/Rural-Low-Income-Worker...,2.78,नवीन क्रोम ओपन करा.
92469,marathi_iitb_microsoft/Rural-Low-Income-Worker...,4.48,सेव्ह अॅज करून नवीन नाव दे.


In [4]:
df['spe']=df['audio_filepath'].apply(lambda x: x.split('_')[4])
df['category']=df['audio_filepath'].apply(lambda x: x.split("_")[2].split('/')[-1])
set(df['spe'])

{'01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'}

In [5]:
df.groupby(['category'])['duration'].sum()/3600

category
College-Students            30.625294
Rural-Low-Income-Workers    34.760367
Urban-Low-Income-Workers    43.888433
Name: duration, dtype: float64

In [6]:
df

,audio_filepath,duration,text,spe,category
0,marathi_iitb_microsoft/College-Students_resamp...,2.68,रामटेक ते वर्धा रेल्वे ने कसं जायचं ?,04,College-Students
1,marathi_iitb_microsoft/College-Students_resamp...,4.78,मला सांग की ऊर्जा ग्लोबल चा ग्राहक निगा क्रमां...,01,College-Students
2,marathi_iitb_microsoft/College-Students_resamp...,4.56,सिंधुदुर्ग पॅसेंजर राजापूर रोड स्टेशन ला कधी य...,10,College-Students
3,marathi_iitb_microsoft/College-Students_resamp...,2.58,आता या नंतर कोणतं गाणं आहे ?,09,College-Students
4,marathi_iitb_microsoft/College-Students_resamp...,6.00,सवारी कार रेंटल्स फरीदाबाद चा तक्रार निवारण क्...,03,College-Students
...,...,...,...,...,...
92466,marathi_iitb_microsoft/Rural-Low-Income-Worker...,7.80,मुंबई एलटीटी - पाटलिपुत्र एक्सप्रेस चे टाइम शे...,02,Rural-Low-Income-Workers
92467,marathi_iitb_microsoft/Rural-Low-Income-Worker...,1.48,शो प्रोफाईल.,03,Rural-Low-Income-Workers
92468,marathi_iitb_microsoft/Rural-Low-Income-Worker...,2.78,नवीन क्रोम ओपन करा.,01,Rural-Low-Income-Workers
92469,marathi_iitb_microsoft/Rural-Low-Income-Worker...,4.48,सेव्ह अॅज करून नवीन नाव दे.,08,Rural-Low-Income-Workers


In [8]:
df.duration.sum()/3600

109.27409444444446

In [9]:
df.columns

Index(['audio_filepath', 'duration', 'text', 'spe', 'category'], dtype='object')

In [7]:
df.groupby(['category','spe'])['duration'].sum()/3600

category                  spe
College-Students          01     3.208583
                          02     2.781606
                          03     3.627822
                          04     2.830500
                          05     3.043756
                          06     3.958094
                          07     2.905900
                          08     2.733728
                          09     2.508444
                          10     3.026861
Rural-Low-Income-Workers  01     4.223589
                          02     3.222744
                          03     4.016050
                          04     4.271828
                          05     4.517817
                          07     2.809050
                          08     4.646111
                          09     3.420094
                          10     3.633083
Urban-Low-Income-Workers  01     5.016561
                          02     4.700572
                          03     0.397361
                          04     2.474828
    

# Train valid test split

In [16]:
valid = df[(df['category'] == 'College-Students') & (df['spe'] == '08') |
        (df['category'] == 'Rural-Low-Income-Workers') & (df['spe'] == '09') |
          (df['category'] == 'Urban-Low-Income-Workers') & (df['spe'] == '04') ]

test = df[(df['category'] == 'College-Students') & (df['spe'] == '09') |
        (df['category'] == 'Rural-Low-Income-Workers') & (df['spe'] == '07') |
          (df['category'] == 'Urban-Low-Income-Workers') & (df['spe'] == '12') ]



In [17]:
test.duration.sum()/3600

7.870977777777777

In [18]:
valid.duration.sum()/3600

8.62865

In [21]:
drop_indexes= list(test.index) + list(valid.index)

In [24]:
train = df.drop(labels=drop_indexes)

In [26]:
train.duration.sum()/3600

92.77446666666665

In [27]:
train.duration.sum()/3600+valid.duration.sum()/3600+ test.duration.sum()/3600

109.27409444444443

In [34]:
valid['audio_filepath'][10].split("/")[1].replace("_resampled","")

'College-Students'

In [39]:
for filename in valid["audio_filepath"][0:5]:
    print("cp "+filename+ " /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/")
    print("cp "+filename.replace(".wav",".txt")+ " /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/")

cp marathi_iitb_microsoft/College-Students_resampled/2938_08_1.wav /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/College-Students/
cp marathi_iitb_microsoft/College-Students_resampled/2938_08_1.txt /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/College-Students/
cp marathi_iitb_microsoft/College-Students_resampled/2554_08_1.wav /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/College-Students/
cp marathi_iitb_microsoft/College-Students_resampled/2554_08_1.txt /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/College-Students/
cp marathi_iitb_microsoft/College-Students_resampled/1954_08_1.wav /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/College-Students/
cp marathi_iitb_microsoft/College-Students_resampled/1954_08_1.txt /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/College-Students/
cp marathi_iitb_microsoft/College-Students_resampled/1734_08_1.w

# Create .sh files for copy

In [40]:
with open("valid.sh","w+") as f:
    for filename in valid["audio_filepath"]:
        print("cp "+filename+ " /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/",file=f)
        print("cp "+filename.replace(".wav",".txt")+ " /home/marathi_finetuning/marathi/data/raw/valid/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/",file=f)
        

In [42]:
with open("test.sh","w+") as f:
    for filename in test["audio_filepath"]:
        print("cp "+filename+ " /home/marathi_finetuning/marathi/data/raw/test/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/",file=f)
        print("cp "+filename.replace(".wav",".txt")+ " /home/marathi_finetuning/marathi/data/raw/test/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/",file=f)
        

In [43]:
with open("train.sh","w+") as f:
    for filename in train["audio_filepath"]:
        print("cp "+filename+ " /home/marathi_finetuning/marathi/data/raw/train/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/",file=f)
        print("cp "+filename.replace(".wav",".txt")+ " /home/marathi_finetuning/marathi/data/raw/train/marathi_iitb_microsoft/"+filename.split("/")[1].replace("_resampled","")+"/",file=f)
        

# Experiments

In [9]:
import re

In [10]:
def remove_punct(text):
    text = re.sub('[%s]' % re.escape("!\।\"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~"), '', text)
    text = text.replace('\u200d', '')
    return text.strip()

# End